In [8]:
from models import *
import pandas as pd
import numpy as np
import os
import sys
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

## Load Training Data

In [2]:
train = pd.read_csv('train_tf_lemmatized_initial_feature_set.csv')
test = pd.read_csv('test_tf_lemmatized_initial_feature_set.csv')
test.head()


,Unnamed: 0,index,text,label,id,score,comp_score,pos_score,000,10,...,yes,yet,york,you,young,younger,your,youth,zero,zombie
0,0,0,"['played', 'well', 'comedy', 'trying', 'randal...",pos,0_10.txt,['I went and saw this movie last night after b...,0.187711,0.142444,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,1,1,"['pressure', 'caddy', 'follows', 'course', 'au...",pos,10000_7.txt,['Actor turned director Bill Paxton follows up...,0.419822,0.178889,0.0,0.0,...,0.0,0.0,0.0,0.045178,0.046412,0.0,0.0,0.0,0.0,0.0
2,2,2,"['intimidated', 'course', 'doe', 'psychologica...",pos,10001_9.txt,"[""As a recreational golfer with some knowledge...",0.225612,0.122000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.056280,0.0,0.0,0.0,0.0,0.0
3,3,3,"['sweet', 'incredibly', 'well', 'doe', 'before...",pos,10002_8.txt,"['I saw this film in a sneak preview, and it i...",0.341656,0.209778,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,4,4,"['caddy', 'kid', 'anti-catholic', 'sympathetic...",pos,10003_8.txt,['Bill Paxton has taken the true story of the ...,0.460417,0.176167,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [3]:
# Shuffle data prior to building model
train = train.sample(frac = 1)

# Make a numeric label for model building (LDA and SVM)
train['numeric_label'] = train.apply(lambda x: 1 if x['label'] == 'pos' else -1, axis = 1)
test['numeric_label'] = test.apply(lambda x: 1 if x['label'] == 'pos' else -1, axis = 1)

train_y = train['numeric_label']
test_y = test['numeric_label']
# Currently only using categorical variables
train_x = pd.DataFrame(train.iloc[:,7:-1])
test_x = pd.DataFrame(test.iloc[:,6:-1])

In [34]:
svm = SVM()
svm.fit(np.array(train_x), np.array(train_y), c=0, epoch=100, rate=.01)
predictions = svm.predict(np.array(test_x))

In [35]:
svm.score(predictions, test_y)

{'accuracy': 0.87876,
 'precision': 0.8713042114343973,
 'recall': 0.8888,
 'f1': 0.8799651498950538}

In [33]:
def svm_grid_search(train_x, train_y, text_x, test_y, c_list, epoch_list, learning_rates,metric='accuracy'):
    parameters = {}
    highest = -1
    best = None
    for c in c_list:
        for epoch in epoch_list:
            for learning_rate in learning_rates:
                key = str(c) + "-" + str(epoch) + "-" + str(learning_rate)
                svm = SVM()
                svm.fit(np.array(train_x), np.array(train_y), c=c, epoch=epoch, rate=learning_rate)
                preditctions = svm.predict(np.array(test_x))
                metrics = {}
                metrics['accuracy'] = accuracy_score(test_y, predictions)
                metrics['precision'] = precision_score(test_y, predictions)
                metrics['recall'] = recall_score(test_y, predictions)
                metrics['f1'] = f1_score(test_y, predictions)
                #metrics = svm.score(predictions, test_y)
                parameters[key] = metrics
                try:
                    if metrics[metric] > highest:
                        highest = metrics[metric]
                        best = key
                except KeyError:
                    raise ValueError(f"Given metric of %s must be one of 'accuracy','precision','recall','f1'" % metric)
    return best, pd.DataFrame(parameters)

In [11]:
cs = [0, .05, .1, 100]#, .2, .5, 1, 3]
es = [1, 10] #30, 50]
lr = [.01]
best, combos = svm_grid_search(train_x, train_y, test_x, test_y, cs, es, lr)

In [12]:
#stanford_train = pd.read_csv('output.csv', nrows=10)
#stanford_train

In [13]:
best

'0-1-0.01'

In [14]:
combos

,0-1-0.01,0-10-0.01,0.05-1-0.01,0.05-10-0.01,0.1-1-0.01,0.1-10-0.01,100-1-0.01,100-10-0.01
accuracy,0.660320,0.660320,0.660320,0.660320,0.660320,0.660320,0.660320,0.660320
precision,0.605208,0.605208,0.605208,0.605208,0.605208,0.605208,0.605208,0.605208
recall,0.922240,0.922240,0.922240,0.922240,0.922240,0.922240,0.922240,0.922240
f1,0.730823,0.730823,0.730823,0.730823,0.730823,0.730823,0.730823,0.730823


In [ ]:
from sklearn.datasets import load_svmlight_file
data = load_svmlight_file("aclImdb/train/labeledBow.feat")
x = data[0]
y = np.array([1 if x >= 5 else -1 for x in data[1]])
svm = SVM()
svm.fit(x, y, epoch=1)

In [ ]:
(x[0]).dot(np.random.rand(x[0].shape[1]))

In [ ]:
x[0].shape

In [ ]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(x, y)
preds = clf.predict(x)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y, preds)